In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = "/content/drive/MyDrive/data/"

SEED = 42 # 시드값

# 데이터 블러오기
train = pd.read_csv(f"{DATA_PATH}titanic_train.csv") # 학습데이터
test = pd.read_csv(f"{DATA_PATH}titanic_test.csv") # 테스트 데이터

# 결측치 처리
age_mean = train["age"].mean()
fare_median = train["fare"].median()
cabin_unk = "UNK"
embarked_mode = train["embarked"].mode()[0]
train["age"] = train["age"].fillna(age_mean)
train["cabin"] = train["cabin"].fillna(cabin_unk)
test["age"] = test["age"].fillna(age_mean)
test["fare"] = test["fare"].fillna(fare_median)
test["cabin"] = test["cabin"].fillna(cabin_unk)
test["embarked"] = test["embarked"].fillna(embarked_mode)

# 특성으로 사용할 변수 선택
cols = ["age","sibsp","parch","fare","pclass","gender","embarked"]
train_ft = train[cols].copy()
test_ft = test[cols].copy()

# 범주형 변수 원핫인코딩
cols = ['gender','embarked']
enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(train[cols])
tmp = pd.DataFrame(
    enc.transform(train_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft,tmp],axis=1).drop(columns=cols)
tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1).drop(columns=cols)

# Min-Max Scaling
scaler = MinMaxScaler()
scaler.fit(train_ft)
train_ft[train_ft.columns] = scaler.transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

# 정답 데이터
target = train["survived"]

train_ft.shape, test_ft.shape, target.shape

Mounted at /content/drive


((916, 10), (393, 10), (916,))

# Model Tuning (Hyperparameter Optimization)
- 학습을 수행하기 전에 설정해야 하는 값인 hyperparameter의 최적값을 탐색

# Manual Search
- 최적의 하이퍼파라미터 값을 직접 탐색하는 방법
- 연구자가 다양한 하이퍼파라미터 조합을 실험하여 어떤 조합이 성능 향상에 좋을지 살펴보고 비교
- 경험에 의한 직관 또는 일반적으로 잘 알려진 하이퍼파라미터 조합으로 탐색하는 방법
- 탐색한 최적의 하이퍼파라미터 조합이 최적이라는 사실을 보장하기가 상대적으로 어려움
- 여러 종류의 하이퍼파라미터를 탐색하고자 할 시 매우 복잡해지는 문제





# Grid Search
- 하이퍼파라미터 별로 다양한 값들을 지정하여 가능한 모든 조합에 대해 성능 결과를 측정한 후 가장 높은 성능을 발휘했던 하이퍼파라미터 조합을 최적값으로 선정하는 방법
- Manual Search에 비해 체계적인 방식이고, Manual Search와 비교하면 좀 더 균등하고 전역적인 탐색이 가능함
- 탐색하고자 하는 하이퍼파라미터의 개수를 여러 종류로 가져갈수록 가능한 모든 조합을 탐색하기 때문에 전체 탐색 시간이 기하급수적으로 증가한다는 단점이 있음
- 각 하이퍼파라미터의 탐색할 값이 10개라고 했을 때
   - 2개의 하이퍼파라미터 조합으로 탐색하면 경우의 수는 100개
   - 3개의 하이퍼파라미터 조합으로 탐색하면 경우의 수는 1000개
   - 기하급수적으로 경우의 수가 늘어남

- GridSearchCV 클래스
    - estimator(첫번째 파라미터)
        - 모델 객체
    - param_grid(두번째 파라미터)
        - 탐색 구간을 딕셔너리로 전달
        - key는 하이퍼파라미터명
        - value는 탐색구간(iterable 객체)
    - scoring
        - 평가지표(문자열로 전달)
    - cv
        - cv 객체 또는 정수

In [2]:
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
model = RandomForestClassifier(random_state=SEED)

# 하이퍼파라미터 탐색 구간
hp = {
    "n_estimators": range(100, 121, 10),
    "criterion": ["gini", "entropy"],
    "max_depth": range(5, 11, 5),
    "min_samples_split": range(2, 5, 2),
    "max_features": ["sqrt", "log2"]
}

grid_search = GridSearchCV(model, hp, cv=cv, scoring="roc_auc", n_jobs=-1)
grid_search.fit(train_ft, target)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(5, 11, 5),
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': range(2, 5, 2),
                         'n_estimators': range(100, 121, 10)},
             scoring='roc_auc')

- 최적의 하이퍼파라미터 조합의 cv 점수

In [3]:
grid_search.best_score_

0.9144457173786703

- 탐색한 최적의 하이퍼파라미터 조합

In [4]:
grid_search.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_split': 4,
 'n_estimators': 100}

- 탐색한 최적의 하이퍼파라미터 조합을 적용하여 학습데이터 학습
- 예측도 가능

In [5]:
pred = grid_search.predict_proba(test_ft)[:,1]
pred[:5]

array([0.17601201, 0.19401594, 0.03364182, 0.92101312, 0.77181654])

- 튜닝한 하이퍼파라미터 조합 저장

In [6]:
import json
with open("rf_hp.json", "w") as f:
    json.dump(grid_search.best_params_, f)

- 튜닝한 하이퍼파라미터 조합 불러오기

In [7]:
with open("rf_hp.json", "r") as f:
    rf_hp = json.load(f)
rf_hp

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_split': 4,
 'n_estimators': 100}

In [8]:
model = RandomForestClassifier(**rf_hp, random_state=SEED)
model.fit(train_ft, target)
pred = model.predict_proba(test_ft)[:,1]
pred.shape

(393,)

# Random Search
- 하이퍼파라미터 별로 탐색 구간을 지정하여 가능한 모든 조합 중에 지정한 횟수만큼 하이퍼파라미터 조합을 랜덤 샘플링하여 성능 결과를 측정한 후 가장 높은 성능을 발휘했던 하이퍼파라미터 조합을 최적값으로 선정하는 방법
- RandomizedSearchCV
    - estimator(첫번째 파라미터)
        - 모델 객체
    - param_distributions(두번째 파라미터)
        - 탐색 구간을 딕셔너리로 전달
        - key는 하이퍼파라미터명
        - value는 탐색구간(iterable 객체)
    - n_iter
        - 탐색 횟수
    - scoring
        - 평가지표(문자열로 전달)
    - cv
        - cv 객체 또는 정수
    - random_state
        - 시드값

In [9]:
from sklearn.model_selection import RandomizedSearchCV

hp = {
    "random_state": [SEED],
    "n_estimators": range(100, 201, 2),
    "criterion": ["gini", "entropy"],
    "max_depth": range(5, 51, 2),
    "min_samples_split": range(2, 21),
    "max_features": ["sqrt", "log2", None]
}

model = RandomForestClassifier()
rand_search = RandomizedSearchCV(model, hp, n_iter=20, cv=cv, scoring="roc_auc", n_jobs=-1, random_state=SEED)
rand_search.fit(train_ft, target)

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=RandomForestClassifier(), n_iter=20, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': range(5, 51, 2),
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_split': range(2, 21),
                                        'n_estimators': range(100, 201, 2),
                                        'random_state': [42]},
                   random_state=42, scoring='roc_auc')

In [10]:
rand_search.best_score_

0.9150661196469378

In [11]:
rand_search.best_params_

{'random_state': 42,
 'n_estimators': 158,
 'min_samples_split': 14,
 'max_features': 'log2',
 'max_depth': 29,
 'criterion': 'entropy'}

In [12]:
rand_search.predict_proba(test_ft)[:,1]

array([2.53822349e-01, 1.98815910e-01, 5.04843263e-02, 8.99572570e-01,
       7.80418400e-01, 4.48759769e-01, 1.26550027e-01, 3.54955833e-01,
       6.07600837e-01, 8.70426721e-01, 3.07510406e-02, 8.78590489e-01,
       8.97221366e-01, 9.09205615e-01, 5.36737244e-01, 1.61208642e-02,
       6.73346312e-02, 0.00000000e+00, 9.53672910e-01, 3.78051873e-03,
       2.32598044e-01, 1.49680373e-01, 9.26325079e-01, 3.99389811e-02,
       9.88907872e-02, 1.31852603e-01, 7.30219136e-02, 1.01069333e-01,
       1.20792054e-01, 1.26031115e-01, 6.80560460e-01, 6.54120392e-01,
       4.72534301e-01, 5.96806418e-02, 7.13897294e-01, 1.50152219e-01,
       9.92319761e-01, 1.20792054e-01, 7.07411308e-02, 2.65835692e-02,
       3.30379009e-03, 3.97250718e-01, 1.50546985e-01, 6.41985642e-03,
       8.94641915e-02, 1.06103474e-01, 5.65313713e-02, 3.64498613e-02,
       6.46900267e-02, 9.77943963e-01, 1.43674485e-01, 4.02584269e-02,
       7.09065343e-01, 8.24702723e-03, 8.94641915e-02, 8.89971219e-01,
      

# 다수 모델을 반복문을 통해 튜닝

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from tqdm.notebook import tqdm

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [14]:
tuning_models = [
    [
        RandomForestClassifier,
        {
            "random_state": [SEED],
            "n_estimators": range(100, 201, 2),
            "criterion": ["gini", "entropy"],
            "max_depth": range(5, 51, 2),
            "min_samples_split": range(2, 21),
            "max_features": ["sqrt", "log2", None]
        }
    ],
    [
        KNeighborsClassifier,
        {
            "n_neighbors": np.arange(3, 21),
            "weights": ["uniform", "distance"],
            "p": [1,2]
        }
    ],
    [
        LogisticRegression,
        {
            "random_state": [SEED],
            "penalty": ["l2", None],
            "C": np.arange(0.1, 1.1, 0.1),
            "max_iter": np.arange(100, 501, 100)
        }
    ],
    [
        XGBClassifier,
        {
            "random_state": [SEED],
            "n_estimators": range(50, 501, 50),
            "learning_rate": np.arange(0.01, 0.51, 0.01),
            "max_depth": range(5, 21),
            "subsample": np.arange(0.5, 1.0, 0.1)
        }
    ],
    [
        LGBMClassifier,
        {
            "random_state": [SEED],
            "n_estimators": range(50, 501, 50),
            "learning_rate": np.arange(0.01, 0.51, 0.01),
            "max_depth": range(5, 21),
            "subsample": np.arange(0.5, 1.0, 0.1)
        }
    ]
]

for model_hp in tqdm(tuning_models):
    model_cls, hp = model_hp
    rand_search = RandomizedSearchCV(
        model_cls(),
        hp,
        n_iter=20,
        cv=cv,
        scoring="roc_auc",
        n_jobs=-1,
        random_state=SEED
    )
    rand_search.fit(train_ft, target)

    model_hp.append(rand_search)

- 확인하기

In [16]:
tuning_models[0]

[sklearn.ensemble._forest.RandomForestClassifier,
 {'random_state': [42],
  'n_estimators': range(100, 201, 2),
  'criterion': ['gini', 'entropy'],
  'max_depth': range(5, 51, 2),
  'min_samples_split': range(2, 21),
  'max_features': ['sqrt', 'log2', None]},
 RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                    estimator=RandomForestClassifier(), n_iter=20, n_jobs=-1,
                    param_distributions={'criterion': ['gini', 'entropy'],
                                         'max_depth': range(5, 51, 2),
                                         'max_features': ['sqrt', 'log2', None],
                                         'min_samples_split': range(2, 21),
                                         'n_estimators': range(100, 201, 2),
                                         'random_state': [42]},
                    random_state=42, scoring='roc_auc')]

In [17]:
for model_cls, _, search_obj in tuning_models:
    print(model_cls.__name__, search_obj.best_score_, search_obj.best_params_)

RandomForestClassifier 0.9150661196469378 {'random_state': 42, 'n_estimators': 158, 'min_samples_split': 14, 'max_features': 'log2', 'max_depth': 29, 'criterion': 'entropy'}
KNeighborsClassifier 0.9086164113990607 {'weights': 'uniform', 'p': 2, 'n_neighbors': 16}
LogisticRegression 0.8977401494806436 {'random_state': 42, 'penalty': 'l2', 'max_iter': 100, 'C': 0.2}
XGBClassifier 0.9084074620172338 {'subsample': 0.5, 'random_state': 42, 'n_estimators': 150, 'max_depth': 6, 'learning_rate': 0.02}
LGBMClassifier 0.9097069857042822 {'subsample': 0.5, 'random_state': 42, 'n_estimators': 150, 'max_depth': 6, 'learning_rate': 0.02}


- 예측하기

In [18]:
pred_list = []
for _, _, search_obj in tuning_models:
    pred = search_obj.predict_proba(test_ft)[:,1]
    pred_list.append(pred)

In [19]:
pred = np.array(pred_list).mean(axis=0)
pred.shape

(393,)

In [20]:
# (pred > 0.5).astype(int)

- 스태킹 앙상블 해보기

In [21]:
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_score

estimators = []

for model_cls, _, search_obj in tuning_models:
    estimators.append(
        (model_cls.__name__, model_cls(**search_obj.best_params_))
    )

params = {
    "estimators": estimators,
    "final_estimator": LogisticRegression(random_state=SEED),
    "cv": cv,
    "n_jobs": -1
}

model = StackingClassifier(**params)
scores = cross_val_score(model, train_ft, target, cv=cv, scoring="roc_auc", n_jobs=-1)
scores.mean()

In [24]:
model.fit(train_ft, target)
model.predict_proba(test_ft)[:,1]

array([0.18376839, 0.15395985, 0.0684144 , 0.89119761, 0.86034983,
       0.44214597, 0.10971056, 0.38374696, 0.6740769 , 0.9050669 ,
       0.07320789, 0.88509153, 0.90995332, 0.92059663, 0.5507118 ,
       0.0743083 , 0.07331629, 0.05905259, 0.9336207 , 0.06675157,
       0.12279897, 0.1199668 , 0.88952434, 0.06785772, 0.08361592,
       0.12579413, 0.0800394 , 0.11583552, 0.1218973 , 0.14470273,
       0.7134648 , 0.7047239 , 0.39693695, 0.07984882, 0.79112989,
       0.12462447, 0.94482348, 0.1218973 , 0.07975294, 0.06951048,
       0.06010404, 0.42720458, 0.11491091, 0.06523177, 0.08154721,
       0.09497519, 0.1119735 , 0.0747938 , 0.08272833, 0.93590065,
       0.11912618, 0.0757771 , 0.77151414, 0.06132294, 0.08154721,
       0.90537664, 0.47604053, 0.23887012, 0.93106069, 0.07517215,
       0.06799164, 0.0768598 , 0.38831619, 0.91595037, 0.06365738,
       0.94362549, 0.1119735 , 0.08717578, 0.65295303, 0.06545882,
       0.11161878, 0.08008793, 0.06193917, 0.94461888, 0.13941

In [25]:
import joblib
joblib.dump(tuning_models, "tuning_models.pkl")

['tuning_models.pkl']

In [26]:
joblib.load("tuning_models.pkl")

[[sklearn.ensemble._forest.RandomForestClassifier,
  {'random_state': [42],
   'n_estimators': range(100, 201, 2),
   'criterion': ['gini', 'entropy'],
   'max_depth': range(5, 51, 2),
   'min_samples_split': range(2, 21),
   'max_features': ['sqrt', 'log2', None]},
  RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                     estimator=RandomForestClassifier(), n_iter=20, n_jobs=-1,
                     param_distributions={'criterion': ['gini', 'entropy'],
                                          'max_depth': range(5, 51, 2),
                                          'max_features': ['sqrt', 'log2', None],
                                          'min_samples_split': range(2, 21),
                                          'n_estimators': range(100, 201, 2),
                                          'random_state': [42]},
                     random_state=42, scoring='roc_auc')],
 [sklearn.neighbors._classification.KNeighborsClassifier,
  {'n_neighbo